In [100]:
import env
import pandas as pd
import numpy as np
from sqlalchemy import text, create_engine
import datetime


Use your function to obtain a connection to the employees database

In [101]:


db = 'employees'
engine = create_engine(env.get_db_url(db))


In [102]:
query = text('select * from employees')
emp_df = pd.read_sql(query, engine.connect())
emp_df

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12
...,...,...,...,...,...,...
300019,499995,1958-09-24,Dekang,Lichtner,F,1993-01-12
300020,499996,1953-03-07,Zito,Baaz,M,1990-09-27
300021,499997,1961-08-03,Berhard,Lenart,M,1986-04-21
300022,499998,1956-09-05,Patricia,Breugel,M,1993-10-13


Read the employees and titles tables into two separate DataFrames.

In [103]:
query = text('select * from titles')
titles_df = pd.read_sql(query, engine.connect())
titles_df

,emp_no,title,from_date,to_date
0,10001,Senior Engineer,1986-06-26,9999-01-01
1,10002,Staff,1996-08-03,9999-01-01
2,10003,Senior Engineer,1995-12-03,9999-01-01
3,10004,Engineer,1986-12-01,1995-12-01
4,10004,Senior Engineer,1995-12-01,9999-01-01
...,...,...,...,...
443303,499997,Engineer,1987-08-30,1992-08-29
443304,499997,Senior Engineer,1992-08-29,9999-01-01
443305,499998,Senior Staff,1998-12-27,9999-01-01
443306,499998,Staff,1993-12-27,1998-12-27


How many rows and columns do you have in each DataFrame? Is that what you expected?

In [104]:
emp_df.shape

(300024, 6)

In [105]:
titles_df.shape

(443308, 4)

Display the summary statistics for each DataFrame.

In [106]:
emp_df.describe()

,emp_no
count,300024.000000
mean,253321.763392
std,161828.235540
min,10001.000000
25%,85006.750000
50%,249987.500000
75%,424993.250000
max,499999.000000


In [107]:
titles_df.describe()

,emp_no
count,443308.000000
mean,253075.034430
std,161853.292613
min,10001.000000
25%,84855.750000
50%,249847.500000
75%,424891.250000
max,499999.000000


How many unique titles are in the titles DataFrame?

In [108]:
len(titles_df['title'].unique())

7

What is the oldest date in the to_date column?

In [109]:
titles_df['to_date'].sort_values().head(1)

16064    1985-03-01
Name: to_date, dtype: object

What is the most recent date in the to_date column?

In [110]:
titles_df['to_date'].max()

datetime.date(9999, 1, 1)

In [111]:
titles_non_current = pd.read_sql("select * from titles where to_date < now()", engine.connect())
titles_non_current.max()

emp_no                 499998
title        Technique Leader
from_date          2002-06-30
to_date            2002-08-01
dtype: object

Exercises II

1. Copy the users and roles DataFrames from the examples above.

In [112]:
# Create the users DataFrame.

users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users


,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


In [113]:
# Create the roles DataFrame

roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles


,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


2. What is the result of using a right join on the DataFrames?


In [114]:
users.merge(roles, left_on='role_id', right_on='id', how='right')

,id_x,name_x,role_id,id_y,name_y
0,1.0,bob,1.0,1,admin
1,2.0,joe,2.0,2,author
2,3.0,sally,3.0,3,reviewer
3,4.0,adam,3.0,3,reviewer
4,NaN,NaN,NaN,4,commenter



3. What is the result of using an outer join on the DataFrames?


In [115]:
users.merge(roles, left_on='role_id', right_on='id', how='outer')

,id_x,name_x,role_id,id_y,name_y
0,1.0,bob,1.0,1.0,admin
1,2.0,joe,2.0,2.0,author
2,3.0,sally,3.0,3.0,reviewer
3,4.0,adam,3.0,3.0,reviewer
4,5.0,jane,NaN,NaN,NaN
5,6.0,mike,NaN,NaN,NaN
6,NaN,NaN,NaN,4.0,commenter



4. What happens if you drop the foreign keys from the DataFrames and try to merge them?


In [116]:
(users.merge(roles, 
            left_on='role_id', 
            right_on='id', 
            how='outer')
    .drop(columns='role_id')
    
)
#basically concats itself


,id_x,name_x,id_y,name_y
0,1.0,bob,1.0,admin
1,2.0,joe,2.0,author
2,3.0,sally,3.0,reviewer
3,4.0,adam,3.0,reviewer
4,5.0,jane,NaN,NaN
5,6.0,mike,NaN,NaN
6,NaN,NaN,4.0,commenter



5. Load the mpg dataset from PyDataset.


In [117]:
from pydataset import data
mpg = data('mpg')
mpg

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact
...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize



6. Output and read the documentation for the mpg dataset.


In [118]:
mpg.info

<bound method DataFrame.info of     manufacturer   model  displ  year  cyl       trans drv  cty  hwy fl  \
1           audi      a4    1.8  1999    4    auto(l5)   f   18   29  p   
2           audi      a4    1.8  1999    4  manual(m5)   f   21   29  p   
3           audi      a4    2.0  2008    4  manual(m6)   f   20   31  p   
4           audi      a4    2.0  2008    4    auto(av)   f   21   30  p   
5           audi      a4    2.8  1999    6    auto(l5)   f   16   26  p   
..           ...     ...    ...   ...  ...         ...  ..  ...  ... ..   
230   volkswagen  passat    2.0  2008    4    auto(s6)   f   19   28  p   
231   volkswagen  passat    2.0  2008    4  manual(m6)   f   21   29  p   
232   volkswagen  passat    2.8  1999    6    auto(l5)   f   16   26  p   
233   volkswagen  passat    2.8  1999    6  manual(m5)   f   18   26  p   
234   volkswagen  passat    3.6  2008    6    auto(s6)   f   17   26  p   

       class  
1    compact  
2    compact  
3    compact  
4    co


7. How many rows and columns are in the dataset?


In [119]:
mpg.shape

(234, 11)


8. Check out your column names and perform any cleanup you may want on them.


In [120]:
mpg.columns

Index(['manufacturer', 'model', 'displ', 'year', 'cyl', 'trans', 'drv', 'cty',
       'hwy', 'fl', 'class'],
      dtype='object')


9. Display the summary statistics for the dataset.


In [121]:
mpg.describe()

,displ,year,cyl,cty,hwy
count,234.000000,234.000000,234.000000,234.000000,234.000000
mean,3.471795,2003.500000,5.888889,16.858974,23.440171
std,1.291959,4.509646,1.611534,4.255946,5.954643
min,1.600000,1999.000000,4.000000,9.000000,12.000000
25%,2.400000,1999.000000,4.000000,14.000000,18.000000
50%,3.300000,2003.500000,6.000000,17.000000,24.000000
75%,4.600000,2008.000000,8.000000,19.000000,27.000000
max,7.000000,2008.000000,8.000000,35.000000,44.000000



10. How many different manufacturers are there?


In [122]:
len(mpg.groupby('manufacturer'))

15


11. How many different models are there?


In [123]:
len(mpg.groupby('model'))

38


12. Create a column named mileage_difference like you did in the DataFrames exercises; this column should contain the difference between highway and city mileage for each car.


In [124]:
mpg['mileage_difference'] = mpg['hwy'] - mpg['cty']
mpg

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,mileage_difference
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10
...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8



13. Create a column named average_mileage like you did in the DataFrames exercises; this is the mean of the city and highway mileage.


In [125]:
mpg['average_mileage'] = (mpg['hwy'] + mpg['cty']) / 2 
mpg

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,mileage_difference,average_mileage
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,23.5
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,25.0
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,25.5
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,25.5
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9,23.5
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8,25.0
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10,21.0
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8,22.0



14. Create a new column on the mpg dataset named is_automatic that holds boolean values denoting whether the car has an automatic transmission.


In [126]:
mpg['is_automatic'] = (mpg.apply(lambda x: x['trans'][0:x['trans'].find('(')], axis=1)) == 'auto'
mpg
#woooowww I did this the hard way

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,mileage_difference,average_mileage,is_automatic
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,23.5,True
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,25.0,False
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,25.5,False
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,25.5,True
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,21.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9,23.5,True
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8,25.0,False
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10,21.0,True
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8,22.0,False



15. Using the mpg dataset, find out which which manufacturer has the best miles per gallon on average?


In [145]:
mpg.groupby(['manufacturer']).mean(['average_mileage']).sort_values(by= ['average_mileage'], ascending=False).head(1)


,displ,year,cyl,cty,hwy,mileage_difference,average_mileage,is_automatic
manufacturer,,,,,,,,
honda,1.711111,2003.0,4.0,24.444444,32.555556,8.111111,28.5,0.444444



16. Do automatic or manual cars have better miles per gallon?

In [150]:
mpg.groupby(['is_automatic']).mean('average_mileage')

,displ,year,cyl,cty,hwy,mileage_difference,average_mileage
is_automatic,,,,,,,
False,2.966234,2002.974026,5.246753,18.675325,25.779221,7.103896,22.227273
True,3.719745,2003.757962,6.203822,15.968153,22.292994,6.324841,19.130573


Exercises III

1. Use your get_db_url function to help you explore the data from the chipotle database.

In [154]:
db = 'chipotle'
engine = create_engine(env.get_db_url(db))

query = text('select * from orders')
chip_df = pd.read_sql(query, engine.connect())
chip_df



,id,order_id,quantity,item_name,choice_description,item_price
0,1,1,1,Chips and Fresh Tomato Salsa,nan,$2.39
1,2,1,1,Izze,[Clementine],$3.39
2,3,1,1,Nantucket Nectar,[Apple],$3.39
3,4,1,1,Chips and Tomatillo-Green Chili Salsa,nan,$2.39
4,5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]",$16.98
...,...,...,...,...,...,...
4617,4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour Cream, Cheese, Lettuce, Guacamole]]",$11.75
4618,4619,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese, Lettuce, Guacamole]]",$11.75
4619,4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto Beans, Guacamole, Lettuce]]",$11.25
4620,4621,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettuce]]",$8.75


In [219]:
chip_df.item_price.astype(str)
chip_df.item_price

0        2.39
1        3.39
2        3.39
3        2.39
4       16.98
        ...  
4617    11.75
4618    11.75
4619    11.25
4620     8.75
4621     8.75
Name: item_price, Length: 4622, dtype: float64

2. What is the total price for each order?


In [209]:
chip_df.columns

Index(['id', 'order_id', 'quantity', 'item_name', 'choice_description',
       'item_price'],
      dtype='object')

In [212]:
chip_df['item_price'].unique()

array([ 2.39,  3.39, 16.98, 10.98,  1.69, 11.75,  9.25,  4.45,  8.75,
       11.25,  8.49,  2.18,  8.99,  1.09,  2.95,  2.15,  3.99, 22.5 ,
       11.48, 17.98, 17.5 ,  4.3 ,  5.9 ,  1.25, 23.78,  6.49, 11.08,
        1.5 , 22.16, 32.94, 22.2 , 10.58,  2.5 , 23.5 ,  7.4 , 18.5 ,
        3.  ,  6.78, 11.89,  9.39,  4.  ,  3.75,  8.69,  2.29,  8.9 ,
        3.27,  3.89,  8.19, 35.  , 27.75, 11.8 ,  6.  , 26.25, 21.96,
        4.36,  7.5 ,  4.78, 13.35,  6.45,  5.07, 22.96,  7.17,  7.98,
        4.5 , 26.07, 12.98, 35.25, 44.25, 10.5 , 33.75, 16.38, 13.52,
        5.  , 15.  ,  8.5 , 17.8 ,  1.99, 11.49])


3. What are the most popular 3 items?


In [220]:
chip_df['item_name'].value_counts().head(3)

item_name
Chicken Bowl           726
Chicken Burrito        553
Chips and Guacamole    479
Name: count, dtype: int64


4. Which item has produced the most revenue?


In [225]:
pd.crosstab(chip_df.item_name,chip_df.item_price, margins=True)

item_price,1.09,1.25,1.5,1.69,1.99,2.15,2.18,2.29,2.39,2.5,...,23.78,26.07,26.25,27.75,32.94,33.75,35.0,35.25,44.25,All
item_name,,,,,,,,,,,,,,,,,,,,,
6 Pack Soft Drink,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,54
Barbacoa Bowl,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,66
Barbacoa Burrito,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,91
Barbacoa Crispy Tacos,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,11
Barbacoa Salad Bowl,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
Barbacoa Soft Tacos,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,25
Bottled Water,22,0,117,0,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,162
Bowl,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
Burrito,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6



5. Join the employees and titles DataFrames together.


In [226]:
db = 'employees'
engine = create_engine(env.get_db_url(db))

query = text('select * from employees')
emp_df = pd.read_sql(query, engine.connect())
emp_df

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12
...,...,...,...,...,...,...
300019,499995,1958-09-24,Dekang,Lichtner,F,1993-01-12
300020,499996,1953-03-07,Zito,Baaz,M,1990-09-27
300021,499997,1961-08-03,Berhard,Lenart,M,1986-04-21
300022,499998,1956-09-05,Patricia,Breugel,M,1993-10-13


In [228]:
query = text('select * from titles')
tit_df = pd.read_sql(query, engine.connect())
tit_df

,emp_no,title,from_date,to_date
0,10001,Senior Engineer,1986-06-26,9999-01-01
1,10002,Staff,1996-08-03,9999-01-01
2,10003,Senior Engineer,1995-12-03,9999-01-01
3,10004,Engineer,1986-12-01,1995-12-01
4,10004,Senior Engineer,1995-12-01,9999-01-01
...,...,...,...,...
443303,499997,Engineer,1987-08-30,1992-08-29
443304,499997,Senior Engineer,1992-08-29,9999-01-01
443305,499998,Senior Staff,1998-12-27,9999-01-01
443306,499998,Staff,1993-12-27,1998-12-27


In [231]:
comb_emp = emp_df.merge(tit_df, left_on='emp_no', right_on='emp_no', how='inner')
comb_emp

,emp_no,birth_date,first_name,last_name,gender,hire_date,title,from_date,to_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26,Senior Engineer,1986-06-26,9999-01-01
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21,Staff,1996-08-03,9999-01-01
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28,Senior Engineer,1995-12-03,9999-01-01
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Engineer,1986-12-01,1995-12-01
4,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Senior Engineer,1995-12-01,9999-01-01
...,...,...,...,...,...,...,...,...,...
443303,499997,1961-08-03,Berhard,Lenart,M,1986-04-21,Engineer,1987-08-30,1992-08-29
443304,499997,1961-08-03,Berhard,Lenart,M,1986-04-21,Senior Engineer,1992-08-29,9999-01-01
443305,499998,1956-09-05,Patricia,Breugel,M,1993-10-13,Senior Staff,1998-12-27,9999-01-01
443306,499998,1956-09-05,Patricia,Breugel,M,1993-10-13,Staff,1993-12-27,1998-12-27



6. For each title, find the hire date of the employee that was hired most recently with that title.


In [237]:
comb_emp.groupby(['title', 'hire_date']).max()

emp_no  birth_date first_name    last_name  \
title              hire_date                                                
Assistant Engineer 1985-02-01   54298  1952-06-29      Taizo       Orsini   
                   1985-02-02  448604  1962-11-23     Yakkov     Poehlman   
                   1985-02-03  442735  1964-04-08   Tetsushi  Waschkowski   
                   1985-02-04  295539  1956-08-05       Shih      Plotkin   
                   1985-02-05  435006  1963-09-10     Shahab     Vernadat   
...                               ...         ...        ...          ...   
Technique Leader   1999-10-30  108697  1963-05-28    Quingbo       Kambil   
                   1999-11-06   11697  1957-12-20     JoAnne        Merey   
                   1999-12-02   73027  1959-11-28       Arto     Spinelli   
                   1999-12-15  441722  1958-12-01    Gonzalo       Aamodt   
                   1999-12-31  294732  1960-11-07     Karlis       Orsini   

                              gender   from_date     to_date  
title              hire_date                                  
Assistant Engineer 1985-02-01      M  1990-12-16  1997-12-16  
                   1985-02-02      M  1998-08-05  9999-01-01  
                   1985-02-03      M  1998-08-23  9999-01-01  
                   1985-02-04      M  1985-02-04  1992-02-05  
                   1985-02-05      M  1999-10-30  9999-01-01  
...                              ...         ...         ...  
Technique Leader   1999-10-30      M  1999-10-30  9999-01-01  
                   1999-11-06      F  1999-12-10  9999-01-01  
                   1999-12-02      M  1999-12-02  9999-01-01  
                   1999-12-15      F  1999-12-15  9999-01-01  
                   1999-12-31      M  2000-01-28  2001-07-04  

[29653 rows x 7 columns]


7. Write the code necessary to create a cross tabulation of the number of titles by department. (Hint: this will involve a combination of SQL code to pull the necessary data and python/pandas code to perform the manipulations.)